<a href="https://colab.research.google.com/github/VellummyilumVinoth/Toxic_Comment_Classification/blob/main/Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import torch

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [54]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [55]:
# Step 1: Install the required libraries
! pip install transformers 

from transformers import DistilBertTokenizer, DistilBertModel,DistilBertForSequenceClassification


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [56]:
from torch import cuda
device = torch.device('cuda' if cuda.is_available() else 'cpu')

print(f"Current device: {device}")

Current device: cpu


In [59]:
# Step 2: Load the fine-tuned DistilBERT model and its tokenizer

model = DistilBertForSequenceClassification.from_pretrained("/content/drive/MyDrive/finetuned_model")
tokenizer = DistilBertTokenizer.from_pretrained('/content/drive/MyDrive/finetuned_distilbert')

OSError: ignored

In [ ]:
# Step 3: Load and preprocess your CSV dataset

import pandas as pd
predict_data = pd.read_csv("/content/drive/MyDrive/preprocessed_Reddit_Data_1.csv")


In [ ]:
predict_data

In [ ]:
columns_to_keep = ['id', 'Author', 'Title']
predict_data = predict_data[columns_to_keep]
predict_data.rename(columns={"Title": "comment_text"}, inplace=True)
titles = predict_data['comment_text'].tolist()

In [ ]:
titles

In [ ]:
threshold = 0.6  # Set your desired threshold value

# Step 4: Tokenize the titles
encoded_inputs = tokenizer(titles, padding=True, truncation=True, return_tensors='pt')

# Step 5: Pass the tokenized inputs through the model to get the predictions
with torch.no_grad():
    outputs = model(**encoded_inputs)
    predicted_labels = outputs.logits




In [ ]:
# Step 6: Map the predicted labels back to their respective class names
class_names = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

predicted_labels = (torch.sigmoid(predicted_labels) > threshold).to(torch.int).tolist()

predicted_labels

In [ ]:
# Create a new DataFrame with the predicted labels
label_df = pd.DataFrame(predicted_labels, columns=class_names)

# Concatenate the original DataFrame with the label DataFrame
result_df = pd.concat([predict_data, label_df], axis=1)

result_df

In [ ]:
# Save the result DataFrame as a new CSV file
result_df.to_csv('/content/drive/MyDrive/outputs/prediction.csv')